# Initialization

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from sklearn.manifold import TSNE
from keras.initializers import glorot_uniform, ones, zeros
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten
from keras.regularizers import l2
import keras.backend as K
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import theano
import theano.tensor as T
!pip install lifelines
from lifelines.utils import concordance_index
!pip install h5py
import h5py
!git clone https://github.com/ThiesCalje/Survival-Analysis-Deep-Learning-AML-.git

In [23]:
import os
os.getcwd()
os.chdir('/content/Survival-Analysis-Deep-Learning-AML-/Data')

# **Import data**

### AML dataset

In [95]:
patient_data_processed = pd.read_excel("Data_10_processed.xlsx",index_col="Sample_ID").sort_index()
patient_data = pd.read_excel("Table 10.xlsx",index_col="Sample_ID").sort_index()
gene_expression = pd.read_excel("Data_10_genex.xlsx").T.sort_index().iloc[:-2,:]
patient_data

,"Treatment_Approach_(AML, ALL, Hybrid)",Treatment_Protocol,Treatment_FLT3_Inhibitor,Transcriptional_Identity,Cohort,Immunophenotype,TARGET_ID,FAB_Designation,Key_Oncogenic_Driver_Event,FLT3_ITD_Allelic_Ratio,FLT3_ITD_Soft_Clip_Reads,Figure_1B_tSNE.x_coordinate,Figure_1B_tSNE.y_coordinate,LSC6_Score,LSC6_Category,Vital_Status,Survival_Days,Follow_Up_Days,EFS_Days,First_Event,SCT_in_First_Remission,Reference
Sample_ID,,,,,,,,,,,,,,,,,,,,,,
SJALL015679_D1,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATMYZ,NaN,PRC2,NaN,NaN,-17.668371,87.973546,0.603193,low,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJALL016438_D1,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATBGC,NaN,T Txn,NaN,NaN,-9.358434,77.480482,1.123439,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJALL016443_D1,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATGZA,NaN,PRC2,NaN,NaN,-2.251928,68.481258,1.264087,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJALL016444_D1,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATKYI,NaN,NUP-r,NaN,NaN,-7.494854,86.200424,0.664466,low,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJALL016445_D1,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATYJK,NaN,PRC2,NaN,NaN,-8.702268,83.301825,1.213566,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SJTALL022649_D2,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PATXKW,NaN,WT1 FLT3 ITD,Not Reported,NA Exon to Exon ITD,-36.284260,100.475237,1.224487,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJTALL170_E,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PARNJB,NaN,NUP-r,NaN,NaN,-12.639045,66.942743,0.983208,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218
SJTALL171_E,ALL,Not Reported,Not Reported,AMTL,ETP,ETP,PARTLY,NaN,FLT3 ITD,Not Reported,0.5042,-37.041095,90.009947,1.641220,medium,NaN,NaN,NaN,NaN,NaN,NaN,Nat Genet. 2017 Aug;49(8):1211-1218


In [50]:
def data_filter(data,keep_variables, one_hot = False):
    T = data['Survival_Days']
    E = data['Vital_Status']

    data_split = {'categorical':["Sample_ID",
                             "Treatment_Approach_(AML, ALL, Hybrid)",
                             "Treatment_Protocol",
                             "Treatment_FLT3_Inhibitor",
                             "Transcriptional_Identity",
                             "Immunophenotype",
                             "FAB_Designation",
                             "Key_Oncogenic_Driver_Event",
                             "LSC6_Category",
                             "First_Event",
                             "SCT_in_First_Remission"],
                 'numerical':["FLT3_ITD_Allelic_Ratio",
                           "FLT3_ITD_Soft_Clip_Reads",
                           "Figure_1B_tSNE.x_coordinate",
                           "Figure_1B_tSNE.y_coordinate",
                           "LSC6_Score",
                           "Vital_Status",
                           "Survival_Days",
                           "Follow_Up_Days",
                           "EFS_Days"],
                'keep_c':[],
                'keep_n':[]
              }

    for variable in keep_variables:

        if variable in data_split['categorical']:
            data_split['keep_c'].append(variable)
            data_split['categorical'].pop(data_split['categorical'].index(variable))

        else:
            data_split['keep_n'].append(variable)
            data_split['numerical'].pop(data_split['numerical'].index(variable))

    data_c = data[data_split['keep_c']]
    data_n = data[data_split['keep_n']]
    if one_hot == True:
        data_c = pd.get_dummies(data_c,dummy_na=True)

    data = data_c.join(data_n)
    return data_c, data_n, data, E, T

In [79]:
def split_data(data, E, split = 0.75):

    censored = data1[E == 0]

    num_patients = len(censored)
    print(num_patients)
    ind_shuffle = random.sample(list(range(num_patients)), num_patients)
    train_size = int(split*num_patients) 
    censored_train = censored.iloc[ind_shuffle][:train_size]
    censored_test = censored.iloc[ind_shuffle][train_size:]

    uncensored = data1[E == 1]

    num_patients = len(uncensored)
    print(num_patients)
    ind_shuffle = random.sample(list(range(num_patients)), num_patients )
    train_size = int(split*num_patients) 
    uncensored_train = uncensored.iloc[ind_shuffle][:train_size]
    uncensored_test = uncensored.iloc[ind_shuffle][train_size:]

    x_train = pd.concat([censored_train, uncensored_train])
    x_test = pd.concat([censored_test, uncensored_test])

    return x_train, x_test

In [82]:
def tSNE(gene_expression, patient_data, var_of_interest, n):

    tsne = TSNE(n, perplexity = 20, verbose=1, random_state = 69)
    tsne_coordinates = tsne.fit_transform(gene_expression)

    return tsne_res

In [84]:
def embedding_TSNE(gene_expression, patient_data, var_of_interest, p, n, weighted = True):

    tsne = TSNE(n, perplexity = 20, verbose=1, random_state = 69)
    tsne_coordinates = tsne.fit_transform(gene_expression)

    patient_categories = patient_data[var_of_interest]
    categories = np.unique(patient_categories)
    clusters = {entity:tsne_coordinates[patient_categories == entity] for entity in categories}

    dimension = len(clusters[list(clusters.keys())[0]][0,:])
    embedding = []
 
    num_colors = len(np.unique(patient_categories))
    colors = plt.get_cmap('rainbow', num_colors)
    
    fig = plt.figure(figsize=(14,10))

    for entity in enumerate(clusters):

        i = entity[0]
        entity = entity[1]

        coords = clusters[entity]
        n_entities = len(coords)
        d = distance_matrix(coords,coords)**p
        

        if weighted == True:
            weights = 1/np.array([np.sum(d[i]) for i in range(n_entities)])
        else:
            weights = np.ones(n_entities)

        com = [sum(coords[:,i]*weights)/sum(weights) for i in range(dimension)]
        embedding.append(com)
        clusters[entity] = clusters[entity], weights, com

        if dimension == 2:
            plt.scatter(coords[:,0],coords[:,1],label=entity,color=colors(i))
            plt.scatter(com[0], com[1], s = 180, color = 'black', marker = 'x')
    
    coordinates = ["X","Y","Z"]
    names = []
    for d in range(dimension):
        names.append(coordinates[d])

    embedding = pd.DataFrame(embedding, index=categories, columns = names)
    plt.legend()
    plt.xlabel('tSNE-one')
    plt.ylabel('tSNE-two')

    return clusters, embedding

In [46]:
keep_variables = ["LSC6_Score","Key_Oncogenic_Driver_Event"]
data_c, data_n, data1, E, T = data_filter(patient_data,keep_variables,one_hot=False)
#print(data_c, data_n, data1)
data1

,Key_Oncogenic_Driver_Event,LSC6_Score
Sample_ID,,
SJAML001408_D1,KMT2Ar,0.204430
SJAML001427_D2,NPM1,-1.127061
SJAML001428_D1,NaN,-1.377134
SJAML001430_D2,NPM1,-0.050137
SJAML001444_D1,KMT2Ar,1.732514
...,...,...
SJMPAL046468_D1,NaN,-0.873553
SJMPAL046469_D1,Ph/Ph Like,0.554643
SJMPAL046470_D1,NaN,0.720019


## Gaussian survival data

In [28]:
h5_file = 'https://github.com/ThiesCalje/Survival-Analysis-Deep-Learning-AML-/blob/main/Data/gaussian_survival_data_1.h5?raw=true'

In [29]:
def normalize(X):
  X = (X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
  return X

In [30]:
with h5py.File("gaussian_survival_data_1.h5", 'r') as f:
  X = f["train"]['x'][()]
  e = f["train"]['e'][()].reshape(-1, 1)
  y = f["train"]['t'][()].reshape(-1, 1)

x_train_gauss = pd.DataFrame(normalize(X))
y_train_gauss = pd.DataFrame(normalize(y))
E_train_gauss = pd.DataFrame(normalize(e))
y_train_gauss

,0
0,0.588690
1,0.330687
2,0.245398
3,1.000000
4,0.929369
...,...
3995,0.420099
3996,1.000000
3997,1.000000
3998,1.000000


# **Auto-encoding**

In [ ]:
def Autoencoder_em(data,em_dim,include_na):

  labels = pd.get_dummies(data,dummy_na=include_na)
  m = len(set(data))
  encoder = Sequential()

  encoder.add(Embedding(input_dim= m, output_dim= em_dim, input_length= m, embeddings_initializer=ones()))
  dif = abs(encoder.get_weights()[0])
  encoder.add(Flatten())
  encoder.add(Dense(m,kernel_initializer=glorot_uniform()))

  encoder.compile(loss= "categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(0.1))

  
  #encoder.summary()

  history = encoder.fit(labels, labels, epochs=100, shuffle=False, verbose=False,validation_split=0.2)
  dif = abs(dif - abs(encoder.get_weights()[0]))
  plot_history(history,0.2)
  pred = encoder.predict(labels)
  embeddings = encoder.get_weights()[0]

  return pred[0]/max(pred[0]), dif

pred, dif = Autoencoder_em(data_c,6, False)
print('prediction = ', '\n', pred)
print('difference = ', '\n', dif)

In [ ]:
def Autoencoder_(data,em_dim,include_na):

  labels = pd.get_dummies(data,dummy_na=include_na)
  m = len(set(data))
  encoder = Sequential()

  encoder.add(Embedding(input_dim= m, output_dim= em_dim, input_length= m, embeddings_initializer=ones()))
  encoder.add(Flatten())
  encoder.add(Dense(m,kernel_initializer=glorot_uniform()))

  encoder.compile(loss= "categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(0.1))

  
  #encoder.summary()

  history = encoder.fit(labels, labels, epochs=300, shuffle=False, verbose=False,validation_split=0.2)
  plot_history(history,0.2)
  pred = encoder.predict(labels)
  embeddings = encoder.get_weights()[0]

  return embeddings, pred[0]/max(abs(pred[0]))

print(Autoencoder(data_c["Key_Oncogenic_Driver_Event"],5, True))

# **Model**

In [33]:
def plot_history(history, validation_split=0):

  plt.plot(history.history['loss'],label='Training loss')

  if validation_split != 0:
    plt.plot(history.history['val_loss'], label='Validation loss')
  plt.title('model loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  #plt.legend()
  
  return 'plot'

In [34]:
def build_and_compile_model(E, T, hid_lay, dropout_rate, input,activation, L2, learning_rate, momemtum, embedding=False):

  model = Sequential()

  if embedding == True:
    print("do something")


  model.add(Dense(hid_lay[0], input_shape=(input,),
                kernel_initializer=glorot_uniform(),
                kernel_regularizer=l2(L2))) # shape= length, dimension
  model.add(Activation(activation))
  model.add(Dropout(dropout_rate))

  if len(hid_lay) != 1:
    for n in hid_lay[1:]:
        model.add(Dense(n,kernel_initializer=glorot_uniform(),
                        kernel_regularizer=l2(L2))) # shape= length, dimension
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))


  model.add(Dense(1, activation="linear",
                    kernel_initializer=glorot_uniform(),
                    kernel_regularizer=l2(L2),
                    activity_regularizer=l2(0.01)))

  model.compile(loss=neg_log_likelihood(E,T),
                optimizer=tf.keras.optimizers.SGD(learning_rate, 
                                                  momentum)
  )
  return model

In [38]:
def neg_log_likelihood(E,T):
    def loss(y_true, y_pred):
        mask = np.ones((len(T), len(T)))
        mask[(y.T - y) > 0] = 0
        hazard_ratio = K.exp(y_pred)*mask
        log_risk = K.log(K.cumsum(hazard_ratio, axis=1)) / np.sum(mask,axis=0)
        uncensored_likelihood = y_pred - log_risk
        censored_likelihood = uncensored_likelihood * E
        neg_likelihood = -K.sum(censored_likelihood)/sum(E)
        return neg_likelihood
    return loss

# **Experiments**

In [36]:
def model_experiment(data, E, T, hidden_layers, dropout_rate, activation, L2, learning_rate, momemtumn ,labels = None, plot = False):
  c = []
  #print(len(labels), data.shape,len(E))
  fig1 = plt.figure(figsize=(10,10))
  for i in range(n):
    input = data.shape[1]
    model = build_and_compile_model(E,
                                    T,
                                    hidden_layers, 
                                    dropout_rate, 
                                    input, 
                                    activation, 
                                    L2, 
                                    learning_rate, 
                                    momentum)
    model.build()
    #model.summary()

    history = model.fit(data, 
                        labels, 
                        epochs=1000, 
                        shuffle=False, 
                        verbose=False,
                        validation_split=0)  # Shuffle False --> Important!!
    if plot == True:
      plot_history(history)
    h_est = model.predict(data)
    #print(h_est[:2])
    c.append(concordance_index(T,-h_est , E))

  fig2 = plt.figure()
  plt.hist(c,bins=10)
  return np.mean(c)
